## Descargar datos

In [2]:
!wget https://gist.githubusercontent.com/trantuyen082001/1fc2f5c0ad1507f40e721e6d18b34138/raw/56c3ca73768ceb74cdf6aa20ee7314c47d6ae08e/heart.csv -O data/original/heart.csv

--2025-10-03 23:51:45--  https://gist.githubusercontent.com/trantuyen082001/1fc2f5c0ad1507f40e721e6d18b34138/raw/56c3ca73768ceb74cdf6aa20ee7314c47d6ae08e/heart.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11019 (11K) [text/plain]
Saving to: ‘data/original/heart.csv’

100%[======================================>] 11,019      --.-K/s   in 0s      

2025-10-03 23:51:45 (49.4 MB/s) - ‘data/original/heart.csv’ saved [11019/11019]



## Leer datos

In [3]:
import pandas as pd

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
df = pd.read_csv("data/original/heart.csv")

In [6]:
last_column = df.pop("output")

In [9]:
df.insert(0, "target", last_column)

,target,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,1,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,1,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,1,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,1,57,0,0,120,354,0,1,163,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0,57,0,0,140,241,0,1,123,1,0.2,1,0,3
299,0,45,1,3,110,264,0,1,132,0,1.2,1,0,3
300,0,68,1,0,144,193,1,1,141,0,3.4,1,2,3
301,0,57,1,0,130,131,0,1,115,1,1.2,1,1,3
